In [1]:
from astropy.table import Table

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
%matplotlib notebook

import numpy as np
import time
from sklearn import neighbors
from vast.voidfinder._voidfinder_cython_find_next import MaskChecker
from vast.voidfinder.distance import z_to_comoving_dist
from vast.voidfinder import ra_dec_to_xyz
import pickle

Update:
- Trying to figure out if we can plot ra v dec, probs not
- If not, needed to figure out how to work with ra_dec_to_xyz() without Rgal variable (Douglass)

In [2]:
mask_file_name = "/Users/lorenzomendoza/Desktop/Research/Function/NSA_main_mask.pickle"

In [3]:
temp_infile = open(mask_file_name, "rb")
mask, mask_resolution = pickle.load(temp_infile)
temp_infile.close()

## V$^2$ void catalog

Generating galaxy array, .npy

In [4]:
V2_galzones = Table.read("/Users/lorenzomendoza/Desktop/Research/Function/V2_REVOLVER-nsa_v1_0_1_galzones.dat",format='ascii.commented_header')
V2_zonevoids = Table.read("/Users/lorenzomendoza/Desktop/Research/Function/V2_REVOLVER-nsa_v1_0_1_zonevoids.dat",format='ascii.commented_header')

V2_gz = np.zeros(len(V2_galzones['zone']),dtype=int)
V2_gz[V2_galzones['zone'] > -1] = 1
'''
for i in range(len(V2_gz)):
    if V2_galzones['zone'][i] > -1:
        #V2_gz[i] = V2_zonevoids['void1'][V2_galzones['zone'][i]]
        V2_gz[i] = 1
''';

In [5]:
V2_galzones[:5]

gal,zone,depth,edge,out
int64,int64,int64,int64,int64
3,1036,0,1,0
7,1036,0,1,0
10,433,3,0,0
12,1036,0,1,0
13,1036,0,1,0


In [8]:
V2_zonevoids[:5]

zone,void0,void1
int64,int64,int64
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4


In [7]:
len(V2_gz)

194125

Extract galaxy void membership. This returns the points in the data set that are V2 voids. 

If returns -1, then not V2

In [6]:
file_name = "/Users/lorenzomendoza/Desktop/Research/Function/V2_nsa_v1_0_1_gal.txt"

In [7]:
data_table_vl = Table.read(file_name, format = "ascii.commented_header")

In [16]:
type(V2_galzones)

astropy.table.table.Table

In [9]:
omega_M = np.float32(0.3)
h = np.float32(1.0)
"""    Parameters
    ==========
    
    z_input : numpy.ndarray of shape (N,)
        redshift values to compute distances for
        
    omega_M : float
        Cosmological matter energy density
        
    h : float
        Hubble constant factor
"""
Rgal = z_to_comoving_dist(data_table_vl['redshift'].astype(np.float32),omega_M,h)
data_table_vl['Rgal'] = Rgal

In [10]:
z_boolean = data_table_vl['redshift']>0 #Edge Case: 513626 = [[-0.  0.  0.]]
data_table_vl = data_table_vl[z_boolean]

In [11]:
galaxies_xyz = ra_dec_to_xyz(data_table_vl)

In [12]:
data_table_vl['x'] = galaxies_xyz[:,0]
data_table_vl['y'] = galaxies_xyz[:,1]
data_table_vl['z'] = galaxies_xyz[:,2]
data_table_vl

index,ra,dec,redshift,rabsmag,Rgal,x,y,z
int64,float64,float64,float64,float64,float32,float64,float64,float64
0,146.6285851269414,-0.7651620976591762,0.0646563172340393,-19.97649574279785,190.97807,-159.47584420504052,105.0408263192713,-2.550359568207288
1,146.9341288729797,-0.6704053610430766,0.12127048522233963,-21.559162139892578,353.4156,-296.1574894141973,192.81135899420556,-4.135143999391673
2,146.9634997327016,-0.5447758299302171,0.055980585515499115,-19.10099220275879,165.68755,-138.89347406859915,90.32432641438558,-1.5753553055354186
3,146.96352370262414,-0.7593404235056136,0.09708637744188309,-20.089942932128906,284.58063,-238.5497230217006,155.13200652067886,-3.771433955733498
4,146.85984630722874,-0.8089065016506019,0.12658987939357758,-21.062517166137695,368.4445,-308.4810128885206,201.40444290203067,-5.201556625836186
5,146.5927838471015,-0.7602574022145185,0.06495923548936844,-19.97614288330078,191.85936,-160.14597434128163,105.62575539512878,-2.5457061490638075
6,146.72838067287842,-0.5569743913807053,0.08931196480989456,-19.900196075439453,262.2765,-219.2735589331227,143.8803771110055,-2.5495592953775956
7,147.18677363814004,-0.4938139466493589,0.09031707048416138,-20.94278907775879,265.1649,-222.8473062347432,143.68810782814072,-2.2853428144724695
8,146.95608041489135,-0.34230852875962586,0.13468021154403687,-21.625717163085938,391.22458,-327.9392589096939,213.3238160439749,-2.337322477587044


In [18]:
V2_galzones

gal,zone,depth,edge,out,x,y,z
int64,int64,int64,int64,int64,float64,float64,float64
3,1036,0,1,0,-238.5497230217006,155.13200652067886,-3.771433955733498
7,1036,0,1,0,-222.8473062347432,143.68810782814072,-2.2853428144724695
10,433,3,0,0,-208.19814649187606,133.92766614907325,-0.15429539345691287
12,1036,0,1,0,-120.02429498525497,76.9669553216956,0.07190305425376976
13,1036,0,1,0,-160.641346636131,106.03878744931015,-3.216040446517236
15,1036,0,1,0,-159.8555305531387,105.75461131763109,-2.829596328341284
22,1036,0,1,0,-206.29328122879542,134.56647604765928,-2.1266625329324707
29,1036,0,1,0,-153.7215735411822,100.54651980616536,-0.6092857630969684
31,1036,0,1,0,-165.55292743628684,111.27335857977052,-2.7613607634454933


In [16]:
V2_galzones['x'] = 0.
V2_galzones['y'] = 0.
V2_galzones['z'] = 0.

for i in range(len(V2_galzones)):
    idx = V2_galzones['gal'][i]
    T2_idx = data_table_vl['index'] == idx
    V2_galzones['x'][i] = data_table_vl['x'][T2_idx]
    V2_galzones['y'][i] = data_table_vl['y'][T2_idx]
    V2_galzones['z'][i] = data_table_vl['z'][T2_idx]


In [23]:
V2_galzones

gal,zone,depth,edge,out,x,y,z
int64,int64,int64,int64,int64,float64,float64,float64
3,1036,0,1,0,-238.5497230217006,155.13200652067886,-3.771433955733498
7,1036,0,1,0,-222.8473062347432,143.68810782814072,-2.2853428144724695
10,433,3,0,0,-208.19814649187606,133.92766614907325,-0.15429539345691287
12,1036,0,1,0,-120.02429498525497,76.9669553216956,0.07190305425376976
13,1036,0,1,0,-160.641346636131,106.03878744931015,-3.216040446517236
15,1036,0,1,0,-159.8555305531387,105.75461131763109,-2.829596328341284
22,1036,0,1,0,-206.29328122879542,134.56647604765928,-2.1266625329324707
29,1036,0,1,0,-153.7215735411822,100.54651980616536,-0.6092857630969684
31,1036,0,1,0,-165.55292743628684,111.27335857977052,-2.7613607634454933


## Remove all galaxies outside the mask

For each galaxy, check to see if it is in the mask.  If it is, keep it!  If it is not, throw it away.
1. Define the variables `rmin` and `rmax`. Values are input by user. 
2. Create a boolean array called `galaxies_boolean`, with the same length as `data_table_vl`, filled with `True` values.
3. Create a `MaskChecker` object using the values `0`, `mask`, `mask_resolution`, `rmin`, and `rmax`.
4. Iterate over each element `i` in `data_table_vl`.
5. Define a variable `curr_gal` as the `i`-th element of the `galaxies_xyz` array reshaped into a 1x3 array.
6. Call the `not_in_mask` method of the `mask_checker` object on `curr_gal`, and assign the result to the variable `not_in_mask`.
7. Set the `i`-th element of the `galaxies_boolean` array to the opposite of the boolean value of `not_in_mask`.
8. After the loop has completed, print the time to run the loop.
9. Filter the `data_table_vl` array using the `galaxies_boolean` array to create a new array called `galaxies_in_mask`.
10. Print the sum of the `True` values in `galaxies_boolean`, the sum of the `False` values in `galaxies_boolean`, the shape of `galaxies_boolean`, and the `galaxies_in_mask` array.


In [18]:
# Stops at: 513625
#514587
#500000 - 513625

In [19]:
# start_time = time.time()

# rmin = 0
# rmax = 312.89816 # Where are these computed?

# galaxies_boolean = np.ones(len(data_table_vl), dtype = bool)

# # print(time.time() - start_time)

# # Declares MaskChecker object from VAST

# mask_checker = MaskChecker(0,
#                         mask,
#                         mask_resolution,
#                         rmin,
#                         rmax,)

# start_time = time.time()
# for i in range(len(data_table_vl)):
    
#     curr_gal = galaxies_xyz[i,:].reshape((1,3))
    
#     not_in_mask = mask_checker.not_in_mask(curr_gal)
    
#     galaxies_boolean[i] = not bool(not_in_mask)

                                    
# print('Time to Run:', time.time() - start_time)
# galaxies_in_mask = data_table_vl[galaxies_boolean]
# # print('\nPoints in Mask Shape:', galaxies_in_mask.shape)

# print('\nSum of Points IN:', np.sum(galaxies_boolean))

# print('\nSum of Points OUT:', np.sum(~galaxies_boolean))

# print('\nBoolean Shape:', galaxies_boolean.shape)

# print('\nPoints in Mask:\n',galaxies_in_mask)


Time to Run: 0.8114590644836426

Sum of Points IN: 349041

Sum of Points OUT: 165357

Boolean Shape: (514398,)

Points in Mask:
 index          ra         ...          y                   z          
------ ------------------ ... ------------------- --------------------
     0  146.6285851269414 ...   105.0408263192713   -2.550359568207288
     2  146.9634997327016 ...   90.32432641438558  -1.5753553055354186
     3 146.96352370262414 ...  155.13200652067886   -3.771433955733498
     5  146.5927838471015 ...  105.62575539512878  -2.5457061490638075
     6 146.72838067287842 ...   143.8803771110055  -2.5495592953775956
     7 147.18677363814004 ...  143.68810782814072  -2.2853428144724695
     9 147.17645340393182 ...  10.195134547011081 -0.11621970176382058
    10 147.24803040379993 ...  133.92766614907325 -0.15429539345691287
    12  147.3295117038904 ...    76.9669553216956  0.07190305425376976
    13  146.5713384071946 ...  106.03878744931015   -3.216040446517236
   ...             

In [20]:
# curr_gal = galaxies_xyz[513625,:]
# print(curr_gal)
# curr_gal = galaxies_xyz[513626,:]
# print(curr_gal)
# type(curr_gal)

# curr_gal = data_table_vl[513625,:]
# print(crr_gal)

#Galaxy 513626 = [[-0.  0.  0.]]

[-203.6687453    46.11162026   -0.45460279]
[-37.61319897   8.52313845   0.29940864]


numpy.ndarray

In [19]:
# xmax = np.min(data_table_vl['x'])
# xmax

-433.95943664150735

In [26]:
# fig = plt.figure(figsize=(12, 12))
# ax = fig.add_subplot(projection='3d')
# ax.scatter(galaxies_in_mask['x'],galaxies_in_mask['y'], galaxies_in_mask['z'],
#            color = 'blue', s=10, alpha=0.5,
#            label='max')
# ax.set(xlabel='X [Mpc/h]',
#        ylabel='Y [Mpc/h]',
#        zlabel='Z [Mpc/h]')

# ax.legend(loc='upper right', fontsize=10)
# plt.title("Galaxy Structure");



<IPython.core.display.Javascript object>

In [20]:
def calc_volume_boundaries(void_cat_A, void_cat_B):
    """Compute the boundaries of the minimal rectangular volume (parallelpiped)
    that completely contains two void catalogs.
    
    Parameters
    ----------
    void_cat_A : astropy.Table
        Table of void data from first catalog.
    void_cat_B : astropy.Table
        Table of void data from second catalog.
        
    Returns
    -------
    x_min : float
    x_max : float
    y_min : float
    y_max : float
    z_min : float
    z_max : float
    """
    x_min = np.minimum(np.min(void_cat_A['x']), np.min(void_cat_B['x']))
    x_max = np.maximum(np.max(void_cat_A['x']), np.max(void_cat_B['x']))
    
    y_min = np.minimum(np.min(void_cat_A['y']), np.min(void_cat_B['y']))
    y_max = np.maximum(np.max(void_cat_A['y']), np.max(void_cat_B['y']))

    z_min = np.minimum(np.min(void_cat_A['z']), np.min(void_cat_B['z']))
    z_max = np.maximum(np.max(void_cat_A['z']), np.max(void_cat_B['z']))

    return x_min, x_max, y_min, y_max, z_min, z_max

def generate_grid_points(x_min, x_max, y_min, y_max, z_min, z_max):
    """Creates a dense rectangular grid of points in 3D for the void volume calculation.
    
    Returns
    -------
    xyz : list
        2D list of points in 3D space.
    """
    
    x_range = np.arange(x_min, x_max) # default spacing: 1 Megaparsec
    y_range = np.arange(y_min, y_max)
    z_range = np.arange(z_min, z_max)


    # Creating a meshgrid from the ranges to 
    X,Y,Z = np.meshgrid(x_range,y_range,z_range)

    x_points = np.ravel(X)
    y_points = np.ravel(Y)
    z_points = np.ravel(Z)
    
    point_coords = np.array([x_points, y_points, z_points])
    
    return point_coords

xmin, xmax, ymin, ymax, zmin, zmax = calc_volume_boundaries(galaxies_in_mask, galaxies_in_mask)

Calls the `calc_volume_boundaries` function on the `galaxies_in_mask` array and assigns the returned values to the variables `xmin`, `xmax`, `ymin`, `ymax`, `zmin`, and `zmax`.

---

pts = generate_grid_points(xmin, xmax, ymin, ymax, zmin, zmax)

Calls the `generate_grid_points` function with the values `xmin`, `xmax`, `ymin`, `ymax`, `zmin`, and `zmax` as arguments and assigns the returned value to the `pts` variable.

---

b = pts.shape
print(b)

Gets the shape of the `pts` array and assigns it to the `b` variable. The shape of the array is printed.


In [21]:
xmin, xmax, ymin, ymax, zmin, zmax = calc_volume_boundaries(V2_galzones, V2_galzones)


#This line makes creates the points in between 
pts = generate_grid_points(xmin, xmax, ymin, ymax, zmin, zmax)

b = pts.shape
print(b)

(3, 64749960)


In [29]:
""" This will take the 3D array of pts and plot along each axis
    (x, y, z)
"""
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')
ax.scatter(pts[0,::100],pts[1,::100], pts[2,::100],
           color = 'blue', s=10, alpha=0.5,
          )
ax.set(xlabel='X [Mpc/h]',
       ylabel='Y [Mpc/h]',
       zlabel='Z [Mpc/h]')

plt.title("Points from Void Volume");

<IPython.core.display.Javascript object>

MaskChecker

In [23]:
start_time = time.time()

rmin = 0
rmax = 312.89816 # Where are these computed?

points_boolean = np.ones(pts.shape[1], dtype = bool)

# print(time.time() - start_time)

# Declares MaskChecker object from VAST

mask_checker = MaskChecker(0,
                        mask,
                        mask_resolution,
                        rmin,
                        rmax,)

start_time = time.time()
for i in range(pts.shape[1]):

    # Current point will iterate through all points at 
    
    curr_pt = pts[:,i].reshape((1,3))
    
    # References MC object to declare if not_in_mask
    
    not_in_mask = mask_checker.not_in_mask(curr_pt)
#     print(not_in_mask, not bool(not_in_mask))

    points_boolean[i] = not bool(not_in_mask)
                                    
points_in_mask = pts[:,points_boolean]
(var, n_points) = points_in_mask.shape
print(time.time() - start_time)
print('\nPoints in Mask Shape:', points_in_mask.shape)

print('\nSum of Points IN:', np.sum(points_boolean))

print('\nSum of Points OUT:', np.sum(~points_boolean))

print('\nBoolean Shape:', points_boolean.shape)

print('\nPoints in Mask:',points_in_mask)


ValueError: Buffer has wrong number of dimensions (expected 2, got 1)

In [24]:
rmin = 0
rmax = 312.89816
def mask_point_filter(pts, mask, mask_resolution, rmin=0, rmax=312.89816):
    start_time = time.time()
    points_boolean = np.ones(pts.shape[1], dtype = bool)

    mask_checker = MaskChecker(0,
                            mask,
                            mask_resolution,
                            rmin,
                            rmax)

    for i in range(pts.shape[1]):
        curr_pt = pts[:,i]
        not_in_mask = mask_checker.not_in_mask(curr_pt)
        points_boolean[i] = not bool(not_in_mask)

    points_in_mask = pts[:,points_boolean]
    print('Time taken:', time.time() - start_time)
    print('Points in Mask Shape:', points_in_mask.shape)
    print('Sum of Points IN:', np.sum(points_boolean))
    print('Sum of Points OUT:', np.sum(~points_boolean))
    print('Boolean Shape:', points_boolean.shape)
    print('Points in Mask:',points_in_mask)
    return points_in_mask, points_boolean


In [25]:
points_in_mask, points_boolean = mask_point_filter(pts, mask, mask_resolution)


ValueError: Buffer has wrong number of dimensions (expected 2, got 1)

In [24]:
def kd_tree(void_cat):
    """We are creating a function to make a KDTree to find the number of points in 
    and out of a catalogue.
    
    Parameters
    ----------
    point_coords: ndarray has a shape of (3,N)
        This is the list of points to query the given void catalogue. N is the number of points given. 
    void_cat: Astropy Table
        This is the given void catalogue.
    
    Returns
    -------
    true_inside: ndarray of shape (N,1)
        Is this the boolean array of length N (same length as point_coords). True means that 1 point 
        is inside the hole.
    """
#############
    cx = void_cat['x']
    cy = void_cat['y']
    cz = void_cat['z']

    sphere_coords = np.array([cx, cy, cz])

    #The .T is meant to transpose the array from (3,1054) to (1054,3)
    sphere_tree = neighbors.KDTree(sphere_coords.T)
    print("KDTree")

##############
    
    return sphere_tree

In [25]:
def point_query(point_coords, sphere_tree, void_cat):
    print("Starting Query")
    #Void cat classifcation
    true_inside = np.zeros(point_coords.shape[1])

    idx = sphere_tree.query(point_coords.T, k = 1, return_distance=False)
    print(idx)
    
    #true_inside = void_cat[idx]
    for i in range(len(idx)):
        true_inside[i] = void_cat[idx[i]]
    
    return true_inside

In [19]:
start_time = time.time()

U = 1

count_in_V1 = np.zeros(U)
count_out_V1 = np.zeros(U)

count_in_V2 = np.zeros(U)
count_out_V2 = np.zeros(U)

inside_both = np.zeros(U)
inside_neither = np.zeros(U)
inside_V1 = np.zeros(U)
inside_V2 = np.zeros(U)

points_in_mask_copy = points_in_mask.copy()

kdTree_V1 = kd_tree(V2_galzones)
kdTree_V2 = kd_tree(V2_galzones)
    
true_inside_V1 = point_query(points_in_mask_copy, kdTree_V1, V2_gz)
print(time.time() - start_time)

count_in_V1[i] = np.sum(true_inside_V1)

    # The "~" inverts the array. So we have true_inside inverted to add up the falses instead of the trues
count_out_V1[i] = np.sum(~true_inside_V1)
    
    # print(time.time() - start_time)

    # total_KIAS = count_in_KIAS + count_out_KIAS

true_inside_V2 = point_query(points_in_mask_copy, kdTree_V2, V2_gz)
print(time.time() - start_time)

count_in_V2[i] = np.sum(true_inside_V2)

    # The "~" inverts the array. So we have true_inside inverted to add up the falses instead of the trues
count_out_V2[i] = np.sum(~true_inside_V2)

    #total_DEL = count_in_DEL + count_out_DEL
    
    # This is the number of points that are inside both A and B
inside_V1_and_V2 = np.logical_and(true_inside_V1, true_inside_V2)
inside_both[i] = np.sum(inside_V1_and_V2)
    
    # This is the number of points that are in neither A and B
not_inside_V1_and_V2 = np.logical_and(~true_inside_V1, ~true_inside_V2)
inside_neither[i] = np.sum(not_inside_V1_and_V2)
    
    # This is the number of points that are in A but not B
inside_v1 = np.logical_and(true_inside_V1, ~true_inside_V2)
inside_V1[i] = np.sum(inside_v1)
    
    # This is the number of points that are not in A but are in B
inside_v2 = np.logical_and(~true_inside_V1, true_inside_V2)
inside_V2[i] = np.sum(inside_v2)

print(time.time() - start_time)
print('\nNumber of points inside V1:', count_in_V1)
print('\nNumber of points outside V2:', count_out_V1)
print('\nNumber of points inside V1:', count_in_V2)
print('\nNumber of points outside V2:', count_out_V2)
print("\nThis is the total number of points: {}".format(n_points))
# print("\nThis is the total number of points in Delaunay: {}".format(total_DEL))

NameError: name 'points_in_mask' is not defined

In [27]:
idx = kdTree_V1.query(points_in_mask_copy.T, k = 1, return_distance=False)

In [30]:
idx.size

23115139

In [31]:
print(V2_gz)
V2_gz.size

[1 1 1 ... 1 1 1]


194125

In [32]:
true_inside = np.zeros(points_in_mask_copy.shape[1])
true_inside.size

23115139

In [34]:
true_inside = np.zeros(points_in_mask_copy.shape[1])

for i in range(len(idx)):
    true_inside[i] = V2_gz[idx[i]]

IndexError: index 428832 is out of bounds for axis 0 with size 194125

In [56]:
len(true_inside)

23115139

In [5]:
test = np.zeros(2000)

array([0., 0., 0., ..., 0., 0., 0.])